## Basic setup and directory structure

In [ ]:
import os

BASE_DIR = '/home/rithomas'
# folder containing MIDI and/or MusicXML files. can have child folders.
RAW_DATA = os.path.join(BASE_DIR, 'data', 'clean_midi')
# TFRecord file that will contain NoteSequence protocol buffers.
SEQUENCES_TFRECORD = os.path.join(BASE_DIR, 'cache', 'improvRNN', 'data', 'notesequences.tfrecord')
# Model config must be one of ['basic_improv', 'attention_improv', 'chord_pitches_improv']
IMPROV_RNN_CONFIG = 'chord_pitches_improv'
# Model directory for storing weights and tensorboard stuff
IMPROV_RNN_DIR = os.path.join(BASE_DIR, 'cache', 'improvRNN', 'model')
# Dataset of Sequence Examples for the ImprovRNN Model
IMPROV_RNN_DATASET = os.path.join(BASE_DIR, 'cache', 'improvRNN')
# Training sequence examples
TRAINING_EXAMPLES = os.path.join(IMPROV_RNN_DATASET, 'training_lead_sheets.tfrecord')
# Dir for testing outputs
OUTPUTS_PATH = os.path.join(IMPROV_RNN_DATASET, 'outputs')
# Final bundled model
BUNDLE_DIR = os.path.join(IMPROV_RNN_DIR, IMPROV_RNN_CONFIG + '.mag')

if not os.path.exists(SEQUENCES_TFRECORD):
    !python3 "magenta/scripts/convert_dir_to_note_sequences.py" \
      --input_dir=$RAW_DATA \
      --output_file=$SEQUENCES_TFRECORD \
      --recursive

## Create ImprovRNN Dataset

In [ ]:
!python3 "magenta/models/improv_rnn/improv_rnn_create_dataset.py" \
    --config=$IMPROV_RNN_CONFIG \
    --input=$SEQUENCES_TFRECORD \
    --output_dir=$IMPROV_RNN_DATASET \
    --eval_ratio=0.10

## Train ImprovRNN on Dataset

In [ ]:
!python3 "magenta/models/improv_rnn/improv_rnn_train.py" \
    --config=$IMPROV_RNN_CONFIG \
    --run_dir=$IMPROV_RNN_DIR \
    --sequence_example_file=$TRAINING_EXAMPLES \
    --hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
    --num_training_steps=20000

## Generate some outputs for testing

In [ ]:
!python3 "magenta/models/improv_rnn/improv_rnn_generate.py" \
    --config=$IMPROV_RNN_CONFIG \
    --run_dir=$IMPROV_RNN_DIR \
    --output_dir=$OUTPUTS_PATH \
    --num_outputs=10 \
    --primer_melody="[57]" \
    --backing_chords="Am Dm G C F Bdim E E" \
    --render_chords

## Save model if you like it!

In [ ]:
!python3 "magenta/models/improv_rnn/improv_rnn_generate.py" \
    --config=$IMPROV_RNN_CONFIG \
    --run_dir=$IMPROV_RNN_DIR \
    --hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
    --bundle_file=$BUNDLE_DIR \
    --save_generator_bundle